# Improved Search for Large Single Document with Highlights

This notebook demonstrates a streamlined approach to document search using Highlights:

1. **Efficiency**: Processes large documents without sending the entire content to a frontier LLM
2. **Precision**: Uses Highlights' contextual awareness to identify relevant passages
3. **Cost-effectiveness**: Reduces token usage by focusing only on relevant sections

The implementation follows these steps:
1. Convert a PDF document to text
2. Split the text into page-level chunks
3. Use Highlights API to retrieve contextually relevant chunks
4. Send only the most relevant chunks to OpenAI for response generation

## Setup

Install the required libraries.

In [ ]:
!pip install openai python-dotenv tiktoken PyPDF2

import os
import openai
import tiktoken
from dotenv import load_dotenv
from base_client import HighlightsClient
from utils import PDFProcessor

## Loading Environment Variables

Create a .env file with your API keys:
```
HIGHLIGHTS_API_KEY=your-highlights-api-key
OPENAI_API_KEY=your-openai-api-key
```

In [6]:
# Load environment variables
load_dotenv()

# Initialize clients
highlights_client = HighlightsClient(api_key=os.getenv('HIGHLIGHTS_API_KEY'))
openai.api_key = os.getenv('OPENAI_API_KEY')

## PDF Processing

For this example we use the 2023 10K filling for UBER with about 169K tokens, which is too large to be directly loaded into GPT4o or Sonnet. We extract the text from the pdf and use a simple chunking at the section level.

In [3]:
# Initialize processor
processor = PDFProcessor(highlights_client)

# Path to your PDF file
pdf_path = 'data/uber_2023_10k.pdf'

# Extract text chunks from PDF
text_chunks = processor.extract_text_from_pdf(pdf_path)

encoding = tiktoken.get_encoding("o200k_base") # gpt-4o tokenizer
num_tokens = len(encoding.encode("".join(text_chunks)))

print(f"Extracted {len(text_chunks)} pages from PDF with a total of {num_tokens} tokens")


Extracted 191 pages from PDF with a total of 169419 tokens


## Contextual Retrieval with Highlights

Unlike vector search that treats chunks in isolation, Highlights evaluates each segment within its surrounding context. This can lead to improvements in retrieval accuracy for complex documents.

In [4]:
query = "What drove revenue change for UBER in FY23?"

# Search for relevant chunks
relevant_chunks = processor.search_relevant_chunks(
    query=query,
    text_chunks=text_chunks,
    top_n=5  # Limiting to top 5 chunks balances completeness with token efficiency
)

print(f"Found {len(relevant_chunks)} relevant chunks")


Found 5 relevant chunks


## Response Generation

By forwarding only the most relevant chunks to a frontier model, we achieve two key benefits:
1. Reduced token consumption and lower costs
2. Higher quality responses by eliminating distracting or irrelevant content

In [7]:
# Generate response using OpenAI
response = processor.generate_response(
    query=query,
    context=relevant_chunks
)

print("\nGenerated Response:")
print(response)


Generated Response:
In FY23, Uber's revenue change was primarily driven by several key factors:

1. **Increase in Mobility Revenue**: Mobility revenue increased by $5.8 billion, or 41%, largely due to a 31% year-over-year increase in Mobility Gross Bookings, which was driven by an increase in trip volumes.

2. **Growth in Delivery Revenue**: Delivery revenue rose by $1.3 billion, or 12%, primarily attributed to a 14% increase in Delivery Gross Bookings, which was driven by higher delivery orders and larger basket sizes.

3. **Decrease in Freight Revenue**: Revenue growth was partially offset by a $1.7 billion decrease in the Freight segment, with Freight Gross Bookings declining 25% year-over-year due to lower revenue per load and volume amid a challenging freight market cycle.

4. **Business Model Changes**: The overall increases in Mobility and Delivery revenue were also negatively impacted by business model changes in some countries, which classified certain sales and marketing cos